In [30]:
# !pip install keras_cv_attention_models

In [1]:
# import matplotlib.pyplot as plt
# import ghostevals
import os
from tensorflow import keras
import tensorflow as tf
# import IJB_evals
# import keras_cv_attention_models
import GhostFaceNets_with_Bias

In [32]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


2 ways to load the model

Stride 1

In [5]:
#Either
# model = keras.models.load_model('..\\src\\models\\ghostFaceNet\\GhostFaceNet_W1.3_S1_ArcFace.h5', compile=False)
model = keras.models.load_model('E:\\taha\\code\\ras-face\\.venv\\src\\models\\ghostFaceNet\\epoch17.h5', compile=False)


In [6]:
base_dir = "..\\src\\models\\ghostFaceNet"
tflite_path = base_dir + "\\tflites"
basic_model_dir = tflite_path + "\\basic_model17.tflite"
quant_model_dir = tflite_path + "\\quant_model17.tflite"
float16_model_dir = tflite_path + "\\float16_model17.tflite"


In [7]:
model.summary()

Model: "GhostNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 112, 112, 3)]        0         []                            
                                                                                                  
 conv2d_53 (Conv2D)          (None, 56, 56, 20)           540       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_80 (Ba  (None, 56, 56, 20)           80        ['conv2d_53[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_48 (PReLU)       (None, 56, 56, 20)           20        ['batch_normalization_8

##### basic convert model

In [8]:
# Convert the model. 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpbghvqmbv\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpbghvqmbv\assets


In [9]:

# Save the model.
print('Saving basic model...')
with open(basic_model_dir, 'wb') as f:
  f.write(tflite_model)

Saving basic model...


##### default optimization (dynamic range quantization)

In [10]:
del converter
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter._experimental_lower_tensor_list_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpdw45w7mp\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpdw45w7mp\assets


In [11]:

# Save the model.
print('Saving quanized model...')
with open(quant_model_dir, 'wb') as f:
  f.write(tflite_model)

Saving quanized model...


##### float_16 optimization

In [12]:
del converter
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter._experimental_lower_tensor_list_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpyetyg3m0\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpyetyg3m0\assets


In [13]:
# Save the model.
print('Saving float_16 model...')
with open(float16_model_dir, 'wb') as f:
  f.write(tflite_model)

Saving float_16 model...


##### keras model test

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import time

# Load the pre-trained model
model = keras.models.load_model('E:\\taha\\code\\ras-face\\.venv\\src\\keras\\content\\keras-facenet\\model\\keras\\model\\facenet_keras.h5', compile=False)
# model = keras.models.load_model('E:\\taha\\code\\ras-face\\.venv\\src\\models\\ghostFaceNet\\epoch17.h5', compile=False)


# Function to preprocess image data
def preprocess_image(image_path):
    # img = image.load_img(image_path, target_size=(112, 112))  # Adjust target size as per your model requirements
    img = image.load_img(image_path, target_size=(160, 160))
    img_array = image.img_to_array(img)
    mean, std = img_array.mean(), img_array.std()
    img_array = (img_array - mean) / std
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_image(image_path, model):
    preprocessed_img = preprocess_image(image_path)
    st = time.time()
    predictions = model.predict(preprocessed_img)
    end = time.time()
    print("Time taken for prediction:", end-st, "Mseconds")
    return predictions

# Path to the image you want to predict
image_path = 'E:\\taha\\code\\ras-face\\.venv\\src\\dataset\\myImages\\train\\taha\\photo_2022-03-06_12-06-03.jpg'
image_path2 = "E:\\taha\\code\\ras-face\\.venv\\src\\dataset\\myImages\\train\\taha\\IMG_1199.JPG"
taha2 = "dataset/apa-dataset/crop/train/taha/photo_2023-10-11_16-48-57-cropped.jpg"
taha1 = "dataset/apa-dataset/crop/train/taha/photo_2023-10-11_16-49-25-cropped.jpg"
ali1 = "dataset/apa-dataset/crop/train/Ali_Hajghani/photo_10_2023-10-11_16-44-44-cf.jpg"
ali2 = "dataset/apa-dataset/crop/train/Ali_Hajghani/photo_10_2023-10-11_16-44-44-crr.jpg"
moslem1 = 'dataset/apa-dataset/crop/train/Moslem_attar/photo_3_2023-10-12_16-14-35-cropped.jpg'
moslem2 = 'dataset/apa-dataset/crop/train/Moslem_attar/photo_3_2023-10-12_16-14-35-cf.jpg'
mohammad1 = "dataset/apa-dataset/crop/train/Mohammad_mohammadi/photo_2023-10-11_17-08-50-cropped.jpg"
mohammad2 = "dataset/apa-dataset/crop/train/Mohammad_mohammadi/photo_2023-10-11_17-08-50-cfn.jpg"
imgs = [image_path, image_path2,taha1,taha2,ali1,ali2,moslem1,moslem2,mohammad1,mohammad2]
for i in range(10):
# Make predictions
    predictions = predict_image(imgs[i], model)

    print(predictions.shape)
# print(predictions)

# Print the predicted class
    predicted_class = np.argmax(predictions)
    print("Predicted class:", predicted_class)


1/1 [==============================] - 2s 2s/step
Time taken for prediction: 1.9329636096954346 Mseconds
(1, 512)
Predicted class: 3
1/1 [==============================] - 0s 84ms/step
Time taken for prediction: 0.15156936645507812 Mseconds
(1, 512)
Predicted class: 17
1/1 [==============================] - 0s 78ms/step
Time taken for prediction: 0.1257953643798828 Mseconds
(1, 512)
Predicted class: 312
1/1 [==============================] - 0s 80ms/step
Time taken for prediction: 0.12530255317687988 Mseconds
(1, 512)
Predicted class: 143
1/1 [==============================] - 0s 80ms/step
Time taken for prediction: 0.12307858467102051 Mseconds
(1, 512)
Predicted class: 79
1/1 [==============================] - 0s 78ms/step
Time taken for prediction: 0.12261533737182617 Mseconds
(1, 512)
Predicted class: 96
1/1 [==============================] - 0s 79ms/step
Time taken for prediction: 0.12317538261413574 Mseconds
(1, 512)
Predicted class: 147
1/1 [==============================] - 0s 7